In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rice = pd.read_csv("./DATA/riceClassification.csv")
rice.head()

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
miss_per = rice.isna().sum() 
miss_per

In [ ]:
# rice.drop(['id'],axis=1,inplace=True)
print(rice.shape)
rice.head()

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10
HIDDEN_NEURONS = 10
LR = 1e-3

In [ ]:
organized = rice.copy()

for col in rice.columns:
  rice[col] = rice[col]/rice[col].abs().max()
  
rice.head()

In [ ]:
X = np.array(rice.iloc[:,:-1])
Y = np.array(rice.iloc[:,-1])

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=101)

In [ ]:
X_test, X_val, Y_test, Y_val = train_test_split(X_test,Y_test,test_size=0.5,random_state=101)

In [ ]:
X_train.shape,X_test.shape,X_val.shape

In [ ]:
print("Training set is: ", X_train.shape[0], " rows which is ", round(X_train.shape[0]/rice.shape[0],4)*100, "%") # Print training shape
print("Validation set is: ",X_val.shape[0], " rows which is ", round(X_val.shape[0]/rice.shape[0],4)*100, "%") # Print validation shape
print("Testing set is: ",X_test.shape[0], " rows which is ", round(X_test.shape[0]/rice.shape[0],4)*100, "%") # Print testing shape

In [ ]:
class dataset(Dataset):
  def __init__(self, X,Y):
    self.X = torch.tensor(X, dtype=torch.float32).to(device)
    self.Y = torch.tensor(Y, dtype=torch.float32).to(device)
    
  def __len__(self):
    return len(self.X)
  
  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]

In [ ]:
training_data = dataset(X_train, Y_train)
validation_data = dataset(X_val, Y_val)
testing_data = dataset(X_test,Y_test)

In [ ]:
train_dataloader = DataLoader(training_data,batch_size=BATCH_SIZE,shuffle=True)
test_dataloader = DataLoader(validation_data,batch_size=BATCH_SIZE,shuffle=True)
val_dataloader = DataLoader(testing_data,batch_size=BATCH_SIZE,shuffle=True)

In [ ]:
HIDDEN_NEURONS = 10

class MyModal(nn.Module):
  def __init__(self):
    super(MyModal, self).__init__()
    self.input_layer = nn.Linear(X.shape[1], HIDDEN_NEURONS)
    self.fc2 = nn.Linear(HIDDEN_NEURONS, HIDDEN_NEURONS)  # Second layer
    self.fc3 = nn.Linear(HIDDEN_NEURONS, 1)  # Third layer
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, x):
    x = self.input_layer(x)
    x = torch.relu(self.fc2(x))  # Activation before third layer
    x = self.sigmoid(self.fc3(x))  # Apply sigmoid to final output
    return x
  
model = MyModal().to(device)

In [ ]:
summary(model,(X.shape[1],))

In [ ]:
criterion = nn.BCELoss()
optimiser = Adam(model.parameters(),lr=1e-3)

In [ ]:
total_loss_train_plot = []
total_loss_validation_plot = []
total_acc_train_plot = []
total_acc_validation_plot = []

for epoch in range(EPOCHS):
  total_acc_train = 0
  total_loss_train = 0
  total_acc_val = 0
  total_loss_val = 0
  ## Training and Validation
  for data in train_dataloader:

    inputs, labels = data

    prediction = model(inputs).squeeze(1)

    batch_loss = criterion(prediction, labels)

    total_loss_train += batch_loss.item()

    acc = ((prediction).round() == labels).sum().item()

    total_acc_train += acc

    batch_loss.backward()
    optimiser.step()
    optimiser.zero_grad()

  ## Validation
  with torch.no_grad():
    for data in val_dataloader:
      inputs, labels = data

      prediction = model(inputs).squeeze(1)

      batch_loss = criterion(prediction, labels)

      total_loss_val += batch_loss.item()

      acc = ((prediction).round() == labels).sum().item()

      total_acc_val += acc

  total_loss_train_plot.append(round(total_loss_train/1000, 4))
  total_loss_validation_plot.append(round(total_loss_val/1000, 4))
  total_acc_train_plot.append(round(total_acc_train/(training_data.__len__())*100, 4))
  total_acc_validation_plot.append(round(total_acc_val/(validation_data.__len__())*100, 4))

  print(f'''Epoch no. {epoch + 1} Train Loss: {total_loss_train/1000:.4f} Train Accuracy: {(total_acc_train/(training_data.__len__())*100):.4f} Validation Loss: {total_loss_val/1000:.4f} Validation Accuracy: {(total_acc_val/(validation_data.__len__())*100):.4f}''')
  print("="*50)

In [ ]:
with torch.no_grad():
  total_loss_test = 0
  total_acc_test = 0
  
  for data in test_dataloader:
    inputs, labels = data
    prediction = model(inputs).squeeze(1)
    batch_loss_test = criterion(prediction,labels).item()
    total_loss_test += batch_loss_test
    
    acc = ((prediction).round() == labels).sum().item()
    total_acc_test += acc
    
print("Testing accuracy: ",round(total_acc_train/(testing_data.__len__())*100, 4)//4)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

axs[0].plot(total_loss_train_plot, label='Training Loss')
axs[0].plot(total_loss_validation_plot, label='Validation Loss')
axs[0].set_title('Training and Validation Loss over Epochs')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].set_ylim([0, 2])
axs[0].legend()

axs[1].plot(total_acc_train_plot, label='Training Accuracy')
axs[1].plot(total_acc_validation_plot, label='Validation Accuracy')
axs[1].set_title('Training and Validation Accuracy over Epochs')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].set_ylim([0, 100])
axs[1].legend()

plt.tight_layout()

plt.show()

In [ ]:
area = 2357/organized['Area'].abs().max()
MajorAxisLength = 81/organized['MajorAxisLength'].abs().max()
MinorAxisLength = 42/organized['MinorAxisLength'].abs().max()
Eccentricity = 32/organized['Eccentricity'].abs().max()
ConvexArea = 12/organized['ConvexArea'].abs().max()
EquivDiameter = 33/organized['EquivDiameter'].abs().max()
Extent = 98/organized['Extent'].abs().max()
Perimeter = 927/organized['Perimeter'].abs().max()
Roundness = 677/organized['Roundness'].abs().max()
AspectRation = 24/organized['AspectRation'].abs().max()
Class = 24/organized['Class'].abs().max()

my_inputs = [area, MajorAxisLength, MinorAxisLength, Eccentricity, ConvexArea, EquivDiameter, Extent, Perimeter, Roundness, AspectRation]

print("="*20)

# Check the expected feature count
expected_features = X.shape[1]

# If missing a feature, manually add a placeholder (e.g., 0.0 or mean value)
if len(my_inputs) < expected_features:
    my_inputs.append(0.0)  # Adjust based on your dataset logic

# Convert to tensor
my_inputs = torch.tensor(my_inputs, dtype=torch.float32).reshape(1, expected_features).to(device)

# Now pass to the model
prediction = model(my_inputs)
print(prediction)
print("Class is:", round(prediction.item()))
